In [1]:
import pandas as pd
import json

In [2]:
df=pd.read_csv('iris.csv')

In [3]:
json=pd.read_json('algoparams_from_ui.json')

In [4]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
df.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [6]:
json

,session_name,session_description,design_state_data
algorithms,test,test,{'RandomForestClassifier': {'model_name': 'Ran...
feature_generation,test,test,"{'linear_interactions': [['petal_length', 'sep..."
feature_handling,test,test,{'sepal_length': {'feature_name': 'sepal_lengt...
feature_reduction,test,test,"{'feature_reduction_method': 'Tree-based', 'nu..."
hyperparameters,test,test,"{'stratergy': 'Grid Search', 'shuffle_grid': T..."
metrics,test,test,"{'optomize_model_hyperparameters_for': 'AUC', ..."
probability_calibration,test,test,{'probability_calibration_method': 'Sigmoid - ...
session_info,test,test,"{'project_id': '1', 'experiment_id': 'kkkk-11'..."
target,test,test,"{'prediction_type': 'Regression', 'target': 'p..."
train,test,test,"{'policy': 'Split the dataset', 'time_variable..."


## 1)	Read the target and type of regression to be run

In [7]:
json['design_state_data']['target']

{'prediction_type': 'Regression',
 'target': 'petal_width',
 'type': 'regression',
 'partitioning': True}

## 2)	Read the features(which are column names in the csv) and figure out what missing imputation needs to be applied and apply that to the columns loaded in a dataframe 

In [8]:
json['design_state_data']['feature_handling']

{'sepal_length': {'feature_name': 'sepal_length',
  'is_selected': True,
  'feature_variable_type': 'numerical',
  'feature_details': {'numerical_handling': 'Keep as regular numerical feature',
   'rescaling': 'No rescaling',
   'make_derived_feats': False,
   'missing_values': 'Impute',
   'impute_with': 'Average of values',
   'impute_value': 0}},
 'sepal_width': {'feature_name': 'sepal_width',
  'is_selected': True,
  'feature_variable_type': 'numerical',
  'feature_details': {'numerical_handling': 'Keep as regular numerical feature',
   'rescaling': 'No rescaling',
   'make_derived_feats': False,
   'missing_values': 'Impute',
   'impute_with': 'custom',
   'impute_value': -1}},
 'petal_length': {'feature_name': 'petal_length',
  'is_selected': True,
  'feature_variable_type': 'numerical',
  'feature_details': {'numerical_handling': 'Keep as regular numerical feature',
   'rescaling': 'No rescaling',
   'make_derived_feats': False,
   'missing_values': 'Impute',
   'impute_with': '

### **as there is not any null value,so imputation is not needed

In [9]:
from sklearn.feature_extraction import FeatureHasher
fh=FeatureHasher(n_features=3,input_type='string')

In [10]:
sp = fh.fit_transform(df['species'])
df1 = pd.DataFrame(sp.toarray(), columns=['fh1', 'fh2', 'fh3'])
new_df=pd.concat([df, df1], axis=1)

In [11]:
new_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species,fh1,fh2,fh3
0,5.1,3.5,1.4,0.2,Iris-setosa,-4.0,2.0,1.0
1,4.9,3.0,1.4,0.2,Iris-setosa,-4.0,2.0,1.0
2,4.7,3.2,1.3,0.2,Iris-setosa,-4.0,2.0,1.0
3,4.6,3.1,1.5,0.2,Iris-setosa,-4.0,2.0,1.0
4,5.0,3.6,1.4,0.2,Iris-setosa,-4.0,2.0,1.0


In [12]:
json['design_state_data']['target']

{'prediction_type': 'Regression',
 'target': 'petal_width',
 'type': 'regression',
 'partitioning': True}

## 3)	Parse the json and make the model objects(using sklean) that can handle what is required in the “prediction_type” specified in the JSON(See #1 where “prediction_type” is specified). Keep in mind not to pick models that don’t apply for the prediction_type specified 

In [13]:
json['design_state_data']['algorithms']

{'RandomForestClassifier': {'model_name': 'Random Forest Classifier',
  'is_selected': False,
  'min_trees': 10,
  'max_trees': 30,
  'feature_sampling_statergy': 'Default',
  'min_depth': 20,
  'max_depth': 30,
  'min_samples_per_leaf_min_value': 5,
  'min_samples_per_leaf_max_value': 50,
  'parallelism': 0},
 'RandomForestRegressor': {'model_name': 'Random Forest Regressor',
  'is_selected': True,
  'min_trees': 10,
  'max_trees': 20,
  'feature_sampling_statergy': 'Default',
  'min_depth': 20,
  'max_depth': 25,
  'min_samples_per_leaf_min_value': 5,
  'min_samples_per_leaf_max_value': 10,
  'parallelism': 0},
 'GBTClassifier': {'model_name': 'Gradient Boosted Trees',
  'is_selected': False,
  'num_of_BoostingStages': [67, 89],
  'feature_sampling_statergy': 'Fixed number',
  'learningRate': [],
  'use_deviance': True,
  'use_exponential': False,
  'fixed_number': 22,
  'min_subsample': 1,
  'max_subsample': 2,
  'min_stepsize': 0.1,
  'max_stepsize': 0.5,
  'min_iter': 20,
  'max_i

In [14]:
X= new_df.drop(labels= 'petal_width', axis= 1)
X=X.drop(labels= 'species', axis= 1)
y= new_df['petal_width']

In [15]:
# Splitting the Dataset 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.33, random_state= 0)

In [16]:
print(f"train size= {X_train.shape},test size= {X_test.shape}")

train size= (100, 6),test size= (50, 6)


## 4)	Run the fit and predict on each model

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
import xgboost as xg
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

lr=LinearRegression()
rfr=RandomForestRegressor(min_samples_leaf=5)
gbr=GradientBoostingRegressor()
rr=Ridge(max_iter=50)
lar=Lasso(max_iter=50)
enr=ElasticNet(max_iter=50)
xgbr=xg.XGBRFRegressor()
dtr=DecisionTreeRegressor()
svr=SVR()
knr=KNeighborsRegressor()



In [18]:
model={lr:'LinearRegression',rfr:'RandomForestRegressor',gbr:'GradientBoostingRegressor',rr:'Ridge',lar:'Lasso',enr:'ElasticNet',xgbr:'XGBRFRegressor',dtr:'DecisionTreeRegressor',svr:'SVR',knr:'KNeighborsRegressor'}
for i,j in model.items():
    reg=i.fit(X_train,y_train)
    pred=i.predict(X_test)
    train_score=reg.score(X_train,y_train)
    test_score=reg.score(X_test,y_test)
    
    print("model: ",j)
    print('train_score: ',train_score.round(3),', test score : ',test_score.round(3))
    print("predicted values : ",pred,'\n')

model:  LinearRegression
train_score:  0.963 , test score :  0.929
predicted values :  [1.95036252 1.14941485 0.31458118 2.15443177 0.24467552 2.2359171
 0.20866187 1.43412076 1.39854877 1.24705383 2.02198282 1.42541611
 1.43536776 1.37205378 1.45315809 0.20040753 1.40845711 1.37369912
 0.16481821 0.2434112  1.91476454 1.46265944 0.37048949 0.18343987
 1.83324456 0.18215822 0.41434181 1.33644713 1.06992122 0.27157752
 2.05672349 1.48086543 0.24425986 1.90483021 2.03025449 1.28428849
 0.30591984 1.5342884  1.35464447 1.23878216 2.00252116 0.18259988
 1.90360053 0.28936785 0.26163452 1.07035421 2.02983016 2.11879914
 1.79851256 2.00337849] 

model:  RandomForestRegressor
train_score:  0.975 , test score :  0.921
predicted values :  [1.82892158 1.25485134 0.26350831 2.04626605 0.23627157 2.18293616
 0.22215953 1.49836786 1.57895626 1.29329852 1.99798451 1.45358994
 1.48947432 1.44476029 1.49037909 0.17626833 1.44064604 1.32249212
 0.18474081 0.26350831 1.81460488 1.40523711 0.27631128 0.1

## 5)	Log to the console the standard model metrics that apply

### ***best fitted model is XGBRFRegressor with train_score:  0.991 , test score :  0.912

In [19]:
xgbr=xg.XGBRFRegressor()
xgboost_regression=xgbr.fit(X_train,y_train)
pred=xgbr.predict(X_test)

In [20]:
from sklearn.metrics import r2_score
print('r2 score: ',r2_score(y_test,pred))

r2 score:  0.9116363107802744
